In [1]:
import pandas as pd
import os
import numpy as np

entorno = 'local'  # Elegir "VM" o "local" para correr en entorno local
nombre_experimento = 'LSTM_producto'
ventana_input = 12
ventana_output = 2
ventana_test = 3
lags = [1, 2, 3, 6, 12]


# Configurar entorno
if entorno == 'VM':
    carpeta_datasets = '~/buckets/b1/datasets'
    carpeta_exp_base = '~/buckets/b1/exp'
elif entorno == 'local':
    carpeta_datasets = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Datos'
    carpeta_exp_base = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Resultados'
else:
    raise Exception("Entorno especificado incorrectamente")

carpeta_exp = os.path.join(carpeta_exp_base, nombre_experimento)
if not os.path.exists(carpeta_exp):
    os.makedirs(carpeta_exp)


In [2]:
# Cargar datos
sell_in = pd.read_csv(os.path.join(carpeta_datasets, 'sell-in.txt'), delimiter='\t')
tb_productos = pd.read_csv(os.path.join(carpeta_datasets, 'tb_productos_descripcion.txt'), delimiter='\t')
tb_stocks = pd.read_csv(os.path.join(carpeta_datasets, 'tb_stocks.txt'), delimiter='\t')
productos_predecir = pd.read_csv(os.path.join(carpeta_datasets, 'productos_a_predecir.txt'), delimiter='\t')



In [3]:

ventas_producto_mes = sell_in.groupby(['periodo', 'product_id'])['tn'].sum()
ventas_producto_mes = ventas_producto_mes.reset_index()
ventas_producto_mes['Timestamp'] = pd.to_datetime(ventas_producto_mes['periodo'], format='%Y%m')
ventas_producto_mes.set_index('Timestamp', inplace=True)
ventas_producto_mes.drop(columns=['periodo'], inplace = True)

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler


def crear_dataset_supervisado(array, input_length, output_length):
    # Inicialización
    X, Y = [], []    # Listados que contendrán los datos de entrada y salida del modelo
    shape = array.shape
    if len(shape) == 1:  # Si tenemos sólo una serie (univariado)
        array = array.reshape(-1, 1)
        cols = 1
    else:  # Multivariado
        fils, cols = array.shape

    # Generar los arreglos (utilizando ventanas deslizantes de longitud input_length)
    for i in range(fils - input_length - output_length + 1):
        X.append(array[i:i + input_length, :].reshape(input_length, cols))
        Y.append(array[i + input_length:i + input_length + output_length, -1].reshape(output_length, 1))

    # Convertir listas a arreglos de NumPy
    X = np.array(X)
    Y = np.array(Y)

    return X, Y

def predecir(x, model, scaler):
    # Calcular predicción escalada en el rango de escalado
    y_pred_s = model.predict(x,verbose=0)
    # Llevar la predicción a la escala original
    y_pred = scaler.inverse_transform(y_pred_s)
    return y_pred.flatten()

def agregar_lags(df, lags):
    for lag in lags:
        df[f'lag_{lag}'] = df['tn'].shift(lag)
    df.bfill(inplace=True)  # Usar backfill para llenar los valores NaN
    return df



lista_productos_LSTM = []
lista_predicciones_LSTM = []
scaler_list = []

for producto in ventas_producto_mes['product_id'].unique():
    if producto in list(productos_predecir['product_id']):
        print(f'Entrenando producto {producto}',  end="\r")
        ventas_mes_por_producto = ventas_producto_mes[ventas_producto_mes['product_id'] == producto].copy()
        #print(f'Tiene {len(ventas_mes_por_producto)} ejemplos de entrenamiento')
        ventas_mes_por_producto.drop(columns=['product_id'], inplace = True)

        #Escalar valor
        scaler = StandardScaler()
        ventas_mes_por_producto['tn'] = scaler.fit_transform(ventas_mes_por_producto)
        scaler_list.append(scaler)
        
        # Agregar lags a los datos escalados y eliminar la columna original
        ventas_mes_por_producto = agregar_lags(ventas_mes_por_producto, lags)
        
        if len(ventas_mes_por_producto) >= (ventana_input + ventana_output):
            #Formatear valores para input LSTM
            X, Y =crear_dataset_supervisado(ventas_mes_por_producto.values, ventana_input, ventana_output)

            # Create and fit the LSTM network
            model = Sequential()
            model.add(LSTM(128, return_sequences=True, input_shape=(ventana_input, X.shape[2]), recurrent_dropout=0.25))
            model.add(LSTM(64,return_sequences = True, recurrent_dropout=0.25))
            model.add(LSTM(32, recurrent_dropout=0.25))
            model.add(Dropout(0.5))
            model.add(Dense(ventana_output))
            model.compile(loss='mean_squared_error', optimizer='adam')
            model.fit(X, Y, epochs=100, batch_size=1, verbose=0)

            #Predecir valores
            input_prediccion = X[-1].reshape(1,X.shape[1],X.shape[2])
            prediccion_mes_2 = predecir(input_prediccion, model, scaler)[1]

            lista_productos_LSTM.append(producto)
            lista_predicciones_LSTM.append(prediccion_mes_2)
        else:
            print('Valores insuficientes para usar ventana de 12 para LSTM, se predice por promedio')
            lista_productos_LSTM.append(producto)
            lista_predicciones_LSTM.append(ventas_mes_por_producto['tn'].mean())

Entrenando producto 20001
Entrenando producto 20002
Entrenando producto 20003
Entrenando producto 20004
Entrenando producto 20005
Entrenando producto 20006
Entrenando producto 20007
Entrenando producto 20008
Entrenando producto 20009
Entrenando producto 20010
Entrenando producto 20011
Entrenando producto 20012
Entrenando producto 20013
Entrenando producto 20014
Entrenando producto 20015
Entrenando producto 20016
Entrenando producto 20017
Entrenando producto 20018
Entrenando producto 20019
Entrenando producto 20020
Entrenando producto 20021
Entrenando producto 20022
Entrenando producto 20023
Entrenando producto 20024
Entrenando producto 20025
Entrenando producto 20026
Entrenando producto 20027
Entrenando producto 20028
Entrenando producto 20029
Entrenando producto 20030
Entrenando producto 20031
Entrenando producto 20033
Entrenando producto 20035
Entrenando producto 20037
Entrenando producto 20038
Entrenando producto 20039
Entrenando producto 20041
Entrenando producto 20042
Entrenando p

In [7]:
resultados_kaggle_LSTM = pd.DataFrame({'product_id': lista_productos_LSTM, 'tn': lista_predicciones_LSTM})
resultados_kaggle_LSTM['tn'] = resultados_kaggle_LSTM['tn'].apply(lambda x: max(0,x))
resultados_kaggle_LSTM.to_csv(os.path.join(carpeta_exp, 'predicciones_LSTM_droput.csv'), index= False)